In [1]:
import os
import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

device = torch.device('cpu')

In [2]:
import cv2

from yolort.utils.image_utils import letterbox, non_max_suppression
from yolort.utils import (
    cv2_imshow,
    get_image_from_url,
    read_image_to_tensor,
    update_module_state_from_ultralytics,
)

## Prepare image and model weights to test

In [3]:
is_half = False

# img_path = "https://raw.githubusercontent.com/zhiqwang/yolov5-rt-stack/master/test/assets/bus.jpg"
img_path = "https://gitee.com/zhiqwang/yolov5-rt-stack/raw/master/test/assets/bus.jpg"  # For user in China
img_raw = get_image_from_url(img_path)

yolort_weight_path = 'yolov5s_r40_updated.pt'
ultralytics_weights_path = "yolov5s.pt"

You can download the weight with following methods

```python
torch.hub.download_url_to_file(
    r'https://github.com/ultralytics/yolov5/releases/download/v4.0/yolov5s.pt',
    ultralytics_weights_path,
    progress=True,
)
```

In [4]:
# Preprocess
img = letterbox(img_raw, new_shape=(640, 640))[0]
img = read_image_to_tensor(img, is_half)
img = img.to(device)

## Load model as ultralytics and inference

In [5]:
conf = 0.25
iou = 0.45

model = torch.hub.load('ultralytics/yolov5', 'custom', path=ultralytics_weights_path)
model = model.to(device)
model.conf = conf  # confidence threshold (0-1)
model.iou = iou  # NMS IoU threshold (0-1)
model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
model.eval()

with torch.no_grad():
    ultralytics_dets = model(img[None])[0]
    ultralytics_dets = non_max_suppression(ultralytics_dets, conf, iou, agnostic=True)[0]

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 v5.0-189-gdaab682 torch 1.8.1+cu102 CUDA:0 (Tesla P100-SXM2-16GB, 16276.25MB)



In [6]:
print(f'Detection results with ultralytics:\n{ultralytics_dets}')

Detection results with ultralytics:
tensor([[3.95028e+02, 2.28687e+02, 4.80437e+02, 5.25319e+02, 8.81427e-01, 0.00000e+00],
        [1.31801e+02, 2.40823e+02, 2.05202e+02, 5.10122e+02, 8.74422e-01, 0.00000e+00],
        [3.28450e+01, 2.39402e+02, 1.42193e+02, 5.31945e+02, 8.50409e-01, 0.00000e+00],
        [1.81174e+01, 1.36144e+02, 4.74266e+02, 4.48792e+02, 7.12929e-01, 5.00000e+00],
        [1.97870e-01, 2.94924e+02, 4.41640e+01, 5.27107e+02, 4.00531e-01, 0.00000e+00]])


## Updating model weights from ultralytics to yolort

In [7]:
model = update_module_state_from_ultralytics(arch='yolov5s',
                                             version='v4.0',
                                             custom_path_or_model=ultralytics_weights_path,
                                             set_fp16=is_half)

torch.save(model.state_dict(), yolort_weight_path)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_v4.0

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                     

## Load model as yolort and inference

In [8]:
from yolort.models.yolo import yolov5_darknet_pan_s_r40 as yolov5s

model = yolov5s(score_thresh=conf, nms_thresh=iou)
model.load_state_dict(torch.load(yolort_weight_path))

# Load model
if is_half:
    model.half()  # to FP16
model.eval()

with torch.no_grad():
    yolort_dets = model(img[None])

In [9]:
print(f"Detection boxes with yolort:\n{yolort_dets[0]['boxes']}")

Detection boxes with yolort:
tensor([[3.95028e+02, 2.28687e+02, 4.80437e+02, 5.25319e+02],
        [1.31801e+02, 2.40823e+02, 2.05202e+02, 5.10122e+02],
        [3.28450e+01, 2.39402e+02, 1.42193e+02, 5.31945e+02],
        [1.81174e+01, 1.36144e+02, 4.74266e+02, 4.48792e+02],
        [1.97876e-01, 2.94923e+02, 4.41640e+01, 5.27107e+02]])


In [10]:
print(f"Detection scores with yolort:\n{yolort_dets[0]['scores']}")

Detection scores with yolort:
tensor([0.88143, 0.87442, 0.85041, 0.71293, 0.40053])


In [11]:
print(f"Detection labels with yolort:\n{yolort_dets[0]['labels']}")

Detection labels with yolort:
tensor([0, 0, 0, 5, 0])


## Varify the detection results between yolort and ultralytics

In [12]:
torch.testing.assert_allclose(yolort_dets[0]['boxes'], ultralytics_dets[:, :4], rtol=1e-05, atol=1e-05)

print("Exported model has been tested, and the result looks good!")

Exported model has been tested, and the result looks good!
